# CNV Random Forest

In [1]:
library(randomForest)
library(dplyr)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘dplyr’

The following object is masked from ‘package:randomForest’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
library(tictoc)

In [3]:
gdsc = readRDS('/home/wangmk/UM/Research/BDSI/Data_Results/gdsc_screened.rds')
cnv = readRDS('/home/wangmk/UM/Research/BDSI/Data_Results/cnNEW.rds')

In [4]:
names(cnv) <- make.names(names(cnv))

In [5]:
get.t <- function(dat, labs){
    # split the data into effective and ineffective
    effect <- dat[labs]
    ineffect <- dat[!labs]
    
    # calculate the two sample means
    effect.bar <- mean(effect)
    ineffect.bar <- mean(ineffect)
    
    # calculate the two sample variances
    v.effect <- var(effect)
    v.ineffect <- var(ineffect)
    
    # calculate the sample sizes
    n.effect <- length(effect)
    n.ineffect <- length(ineffect)
    
    # calculate the sd
    s <- sqrt((v.effect/n.effect) + (v.ineffect/n.ineffect))
    
    # calculate the test statistic
    T <- (effect.bar - ineffect.bar)/s
    
    # calculate the degrees of freedom
    # df = ((v.effect/n.effect+v.ineffect/n.ineffect)^2)/(v.effect^2/(n.effect^2 * (n.effect-1))+v.ineffect^2/(n.ineffect^2 * (n.ineffect-1)))
    
    # compare our t value and the threshold, decide whether we should keep it or not
    return(abs(T))
}

In [6]:
outcome <- data.frame(Gene = colnames(cnv))

In [7]:
tic()
codes <- c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")
resultmat = matrix(nrow=12,ncol=6)
rownames(resultmat) = c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")
colnames(resultmat) = c("10","50","100","200","500","1000")
for (code in codes){
  gdsc_sub <- subset(gdsc, DRUG_ID_lib == code)[,c("CL","EFFECT")]
  cnv_sub <- cnv[as.character(gdsc_sub$CL),]
  cnv_sub_full <- cbind(cnv_sub, gdsc_sub['EFFECT'])
  # outcome$T <- apply(cnv_sub,2,get.t,gdsc_sub$EFFECT)
  n_features = c(50,100,200,500,1000)
  train_accuracy = c()
  test_accuracy = c()
  for (n_feature in n_features){
    train_error = c()
    test_error = c()
    for (j in 1:8){
      set.seed(j)
      train_id=sample(1:nrow(cnv_sub_full),0.7*nrow(cnv_sub_full))
      train_cnv = cnv_sub_full[train_id,]
      test_cnv = cnv_sub_full[-train_id,]
      outcome$T <- apply(train_cnv[,-ncol(train_cnv)], 2, get.t, train_cnv$EFFECT)
      subset_id = outcome %>% top_n(n_feature,T) %>% select(Gene)
      subset_id = unname(t(subset_id[,1]))
      train_cnv_reduced = cbind(train_cnv[,subset_id],train_cnv['EFFECT'])
      test_cnv_reduced = cbind(test_cnv[,subset_id],test_cnv['EFFECT'])
      train_cnv_reduced = as.data.frame(train_cnv_reduced)
      test_cnv_reduced = as.data.frame(test_cnv_reduced)
      #cn.col.name <- gsub("[-]", ".", colnames(train_cnv_reduced))
      #colnames(train_cnv_reduced) <- cn.col.name
      #colnames(test_cnv_reduced) <- cn.col.name
      train_cnv_reduced$EFFECT <- as.factor(train_cnv_reduced$EFFECT)
      test_cnv_reduced$EFFECT <- as.factor(test_cnv_reduced$EFFECT)
      myforest = randomForest(EFFECT~.,data=train_cnv_reduced,mtry=round(sqrt(ncol(train_cnv_reduced)-1)),ntree=50)
      #train_pred = predict(myforest,train_cnv)
      test_pred = predict(myforest,test_cnv_reduced)
      #train_error[j] = mean(train_pred!=train_cnv[['EFFECT']])
      test_error[j] = mean(test_pred!=test_cnv_reduced[['EFFECT']])
  }
  resultmat[code,toString(n_feature)] = 1-mean(test_error)
  }
}
toc()

1687.065 sec elapsed


In [10]:
saveRDS(resultmat,'CNV_RF.rds')

In [42]:
summary(train_cnv_reduced$`37865`)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   2.000   2.000   2.484   3.000   6.000 

In [39]:
train_cnv_reduced$EFFECT = as.factor(train_cnv_reduced$EFFECT)
summary(train_cnv_reduced$EFFECT)

FALSE  TRUE 
   51   137

In [20]:
summary(cnv_sub_full$EFFECT)

   Mode   FALSE    TRUE 
logical      71     198 

In [32]:
summary(train_cnv_reduced$`RP11-242G20.2`)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   2.000   2.000   2.548   3.000   6.000 

In [30]:
train_cnv_reduced[1:6,1:6]

,C10orf35,RP11-242G20.2,AC128709.2,AREG,RP11-44F21.2,RP11-44F21.3
906832,3,3,5,3,3,3
908136,2,2,2,2,2,2
910905,2,2,3,2,2,2
1327775,2,2,3,2,2,2
905955,2,2,3,2,2,2
1327766,2,2,3,3,3,3


In [35]:
cn.col.name <- gsub("[-]", ".", colnames(train_cnv_reduced))
head(cn.col.name)

[1] "C10orf35"      "RP11.242G20.2" "AC128709.2"    "AREG"         
[5] "RP11.44F21.2"  "RP11.44F21.3"

In [36]:
colnames(train_cnv_reduced) <- cn.col.name
train_cnv_reduced[1:5,1:5]

,C10orf35,RP11.242G20.2,AC128709.2,AREG,RP11.44F21.2
906832,3,3,5,3,3
908136,2,2,2,2,2
910905,2,2,3,2,2
1327775,2,2,3,2,2
905955,2,2,3,2,2


In [29]:
myforest = randomForest(EFFECT~.,data=train_cnv_reduced,mtry=round(sqrt(ncol(train_cnv_reduced)-1)),ntree=50)

ERROR: Error in eval(predvars, data, env): object 'RP11-242G20.2' not found


In [ ]:
outcome = tbl_df(outcome)
    subset_id = outcome %>% top_n(n_feature,T) %>% select(Gene)
    subset_id = unname(t(subset_id[,1]))
    cnv_sub_reduced = cnv_sub[,subset_id]
    cnv_final = cbind(cnv_sub_reduced,gdsc_sub$EFFECT)
    cnv_final = as.data.frame(cnv_final)
    colnames(cnv_final)[ncol(cnv_final)]='EFFECT'
    cnv_final$EFFECT = as.factor(cnv_final$EFFECT)
    effective_cnv = cnv_final %>% filter(EFFECT==TRUE)
    ineffective_cnv = cnv_final %>%  filter(EFFECT==FALSE)